In [20]:
import sys  
sys.path.insert(0, '../')
import main as m
# reload
import importlib as i
i.reload(m)
sys.path.insert(0, '')
import copy

In [ ]:
i.reload(m)

In [6]:
PATH = {}
PATH["weather"] = "weather/"
PATH["CO"] = ["orig_CO_2019_ym.nc","orig_CO_2020_ym.nc", "orig_CO_2021_ym.nc"]

In [140]:
FOCUS = "CO"
res = []
c = 0
df_Final = m.pd.DataFrame()
LISTOFVALUES= []
for i in PATH[FOCUS]:
    print(i)
    DS = m.xr.open_dataset(PATH["weather"]+i)

    # write to csv
    # DS.to_dataframe().to_csv("output_filename.csv")
    # read in
    df = DS.to_dataframe()
    df = m.gpd.GeoDataFrame(
        df, geometry=m.gpd.points_from_xy(df.lon, df.lat))

    # convert values
    df["lon"] = df["lon"].astype(float)
    df["lat"] = df["lat"].astype(float)
    # double conversation needs
    df.CO = df.CO.astype(float).astype(int)
    # important! 
    df.crs = "EPSG:23700" #Hungary
    #print(df.head())
    # filter, since there are duplications
    s = list(set(df["Times_bnds"]))
    print(s[0])
    df = df[df["Times_bnds"] == s[0]]
    ###########################################
    ###########################################
    # filter focusing on Hungary
    df = df[df["lon"] < 23.02]
    df = df[df["lon"] > 16.072]
    df = df[df["lat"] > 45.699]
    df = df[df["lat"] < 48.7]

    # since there are only points, expand them to square
    for key in df.index:
        df.geometry[key] = m.Polygon(((df.lon[key]-0.1, df.lat[key]),
                    (df.lon[key], df.lat[key]),
                   (df.lon[key], df.lat[key]+0.1),

                   (df.lon[key]-0.1, df.lat[key]+0.1)))
    
    
    #print(df[FOCUS].tolist())
    if c == 0:
        dfFinal = copy.deepcopy(df)
        dfFinal.rename(columns={str(FOCUS): str(FOCUS+"_"+str(c))}, inplace=True)
        #print(df[FOCUS].tolist())
        LISTOFVALUES.append(df[FOCUS].tolist())
    else:
        l = copy.deepcopy(df[FOCUS].tolist())
        dfFinal[FOCUS+"_"+str(c)] = copy.deepcopy(l)
        LISTOFVALUES.append(l)
    
    c += 1
LISTMATRIX = np.column_stack(LISTOFVALUES)
AVG = list(map(int, np.mean(LISTMATRIX, axis=1).tolist()))
#MAX = np.max(LISTMATRIX, axis=1)
dfFinal['MAX'] = np.max(LISTMATRIX, axis=1).tolist()
dfFinal['AVG'] = AVG

orig_CO_2019_ym.nc
20190101.0
orig_CO_2020_ym.nc
20200101.0
orig_CO_2021_ym.nc
20210101.0


In [141]:
dfFinal


Times_bnds   lon   lat  CO_0  \
lev Times        bnds south_north west_east                                 
1.0 2.019070e+07 0    7           21         20190101.0  16.1  45.7   178   
                                  22         20190101.0  16.2  45.7   173   
                                  23         20190101.0  16.3  45.7   168   
                                  24         20190101.0  16.4  45.7   167   
                                  25         20190101.0  16.5  45.7   165   
...                                                 ...   ...   ...   ...   
                      36          86         20190101.0  22.6  48.6   151   
                                  87         20190101.0  22.7  48.6   151   
                                  88         20190101.0  22.8  48.6   151   
                                  89         20190101.0  22.9  48.6   150   
                                  90         20190101.0  23.0  48.6   149   

                                                                                      geometry  \
lev Times        bnds south_north west_east                                                      
1.0 2.019070e+07 0    7           21         POLYGON ((16.000 45.700, 16.100 45.700, 16.100...   
                                  22         POLYGON ((16.100 45.700, 16.200 45.700, 16.200...   
                                  23         POLYGON ((16.200 45.700, 16.300 45.700, 16.300...   
                                  24         POLYGON ((16.300 45.700, 16.400 45.700, 16.400...   
                                  25         POLYGON ((16.400 45.700, 16.500 45.700, 16.500...   
...                                                                                        ...   
                      36          86         POLYGON ((22.500 48.600, 22.600 48.600, 22.600...   
                                  87         POLYGON ((22.600 48.600, 22.700 48.600, 22.700...   
                                  88         POLYGON ((22.700 48.600, 22.800 48.600, 22.800...   
                                  89         POLYGON ((22.800 48.600, 22.900 48.600, 22.900...   
                                  90         POLYGON ((22.900 48.600, 23.000 48.600, 23.000...   

                                             CO_1  CO_2  MAX  AVG  
lev Times        bnds south_north west_east                        
1.0 2.019070e+07 0    7           21          178   181  181  179  
                                  22          173   177  177  174  
                                  23          169   173  173  170  
                                  24          167   172  172  168  
                                  25          165   170  170  166  
...                                           ...   ...  ...  ...  
                      36          86          151   163  163  155  
                                  87          151   152  152  151  
                                  88          150   150  151  150  
                                  89          149   152  152  150  
                                  90          148   155  155  150  

[2100 rows x 9 columns]

In [134]:
dfFinal['MAX'] = MAX.tolist()
dfFinal['AVG'] = AVG
#dfFinal['AVG'] = dfFinal['AVG'].astype(int)

In [137]:
dfFinal

Times_bnds   lon   lat  CO_0  \
lev Times        bnds south_north west_east                                 
1.0 2.019070e+07 0    7           21         20190101.0  16.1  45.7   178   
                                  22         20190101.0  16.2  45.7   173   
                                  23         20190101.0  16.3  45.7   168   
                                  24         20190101.0  16.4  45.7   167   
                                  25         20190101.0  16.5  45.7   165   
...                                                 ...   ...   ...   ...   
                      36          86         20190101.0  22.6  48.6   151   
                                  87         20190101.0  22.7  48.6   151   
                                  88         20190101.0  22.8  48.6   151   
                                  89         20190101.0  22.9  48.6   150   
                                  90         20190101.0  23.0  48.6   149   

                                                                                      geometry  \
lev Times        bnds south_north west_east                                                      
1.0 2.019070e+07 0    7           21         POLYGON ((16.000 45.700, 16.100 45.700, 16.100...   
                                  22         POLYGON ((16.100 45.700, 16.200 45.700, 16.200...   
                                  23         POLYGON ((16.200 45.700, 16.300 45.700, 16.300...   
                                  24         POLYGON ((16.300 45.700, 16.400 45.700, 16.400...   
                                  25         POLYGON ((16.400 45.700, 16.500 45.700, 16.500...   
...                                                                                        ...   
                      36          86         POLYGON ((22.500 48.600, 22.600 48.600, 22.600...   
                                  87         POLYGON ((22.600 48.600, 22.700 48.600, 22.700...   
                                  88         POLYGON ((22.700 48.600, 22.800 48.600, 22.800...   
                                  89         POLYGON ((22.800 48.600, 22.900 48.600, 22.900...   
                                  90         POLYGON ((22.900 48.600, 23.000 48.600, 23.000...   

                                             CO_1  CO_2  MAX  AVG  
lev Times        bnds south_north west_east                        
1.0 2.019070e+07 0    7           21          178   181  181  179  
                                  22          173   177  177  174  
                                  23          169   173  173  170  
                                  24          167   172  172  168  
                                  25          165   170  170  166  
...                                           ...   ...  ...  ...  
                      36          86          151   163  163  155  
                                  87          151   152  152  151  
                                  88          150   150  151  150  
                                  89          149   152  152  150  
                                  90          148   155  155  150  

[2100 rows x 9 columns]

In [123]:
AVG.tolist(int)

TypeError: function takes exactly 0 arguments (1 given)

In [125]:
dfFinal['AVG'].astype(int)

lev  Times         bnds  south_north  west_east
1.0  2.019070e+07  0     7            21           179
                                      22           174
                                      23           170
                                      24           168
                                      25           166
                                                  ... 
                         36           86           155
                                      87           151
                                      88           150
                                      89           150
                                      90           150
Name: AVG, Length: 2100, dtype: int32